In [1]:
import pandas as pd
import sqlite3
import os

# Parseando DB Dump

Esse notebook tem por objetivo realizar o ETL dos dados fornecidos para o desafio.

Os dados foram fornecidos como um database dump de uma base de dados Maria DB.

Nosso objetivo é criar um banco de dados sqlite e estruturá-lo no mesmo schema da base original MariaDB.

Em seguida, devemos carregar os dados do dump para a base de dados sqlite criada.

Como o sqlite e a MariaDB implementam idiomas SQL distintos, além do fato do conjunto de operações/funcionalidades disponível para o sqlite ser menor do que par ao MariaDB, alguns ajustes no dump serão necessários.

#### Gerenciando diretórios

Nas células abaixo, fazemos o gerenciamento dos diretórios e paths de arquivo utilizados neste script.

In [2]:
base_path = os.path.abspath(os.getcwd())
original_data_path = os.path.join(base_path, 'original_data')

In [3]:
os.listdir(original_data_path)

['Desafio para o processo seletivo.docx',
 'respostas_diagnostico.sql',
 'respostas_diagnostico_dicionario.xlsx',
 '~$safio para o processo seletivo.docx']

In [4]:
data_path = os.path.join(base_path, 'data')

if not os.path.exists(data_path):
    os.mkdir(data_path)

In [5]:
db_path = os.path.join(data_path, 'desafio_selecao.db')

#### Inspecionando os dados

Nas células abaixo inspecionamos o dicionário de dados e o dump de sql.



In [6]:
dicionario = pd.read_excel(os.path.join(
                            original_data_path, 'respostas_diagnostico_dicionario.xlsx'),
                          header=1)

In [7]:
dicionario

,nome do campo,tipo,aceita nulos?,descrição
0,id_resposta,INT(11),NÃO,Chave única para cada resposta do formulário
1,ano_diagnostico,INT(11),NÃO,Ano de realização da pesquisa
2,data_submissao,DATETIME,SIM,Data de envio da resposta
3,orgao,TEXT,SIM,Selecione o Órgão onde trabalha:
4,tipo_orgao,TEXT,SIM,Classificação quanto ao tipo de órgão (Pode se...
5,qtd_equipe,"DECIMAL(10,0)",SIM,Quantas pessoas trabalham de forma dedicada à ...
6,utiliza_metodologia,INT(1),SIM,É utilizada alguma metodologia de gerenciament...
7,desktop_proprio,TEXT,SIM,Preencha os ítens abaixo de acordo com os quan...
8,desktop_locado,TEXT,SIM,Preencha os ítens abaixo de acordo com os quan...
9,desktop_proprio_antigo,TEXT,SIM,Do total de equipamentos PRÓPRIOS informados n...


In [8]:
dump_path = os.path.join(original_data_path, 'respostas_diagnostico.sql')

In [9]:
with open(dump_path, 'r') as f:
    t = f.read(2000)

In [10]:
print(t)

-- --------------------------------------------------------
-- Servidor:                     localhost
-- VersÃ£o do servidor:           10.4.6-MariaDB-1:10.4.6+maria~bionic - mariadb.org binary distribution
-- OS do Servidor:               debian-linux-gnu
-- HeidiSQL VersÃ£o:              10.2.0.5599
-- --------------------------------------------------------

/*!40101 SET @OLD_CHARACTER_SET_CLIENT=@@CHARACTER_SET_CLIENT */;
/*!40101 SET NAMES utf8 */;
/*!50503 SET NAMES utf8mb4 */;
/*!40014 SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS, FOREIGN_KEY_CHECKS=0 */;
/*!40101 SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE='NO_AUTO_VALUE_ON_ZERO' */;



-- Copiando estrutura do banco de dados para desafio_selecao
CREATE DATABASE IF NOT EXISTS `desafio_selecao` /*!40100 DEFAULT CHARACTER SET utf8 COLLATE utf8_unicode_ci */;
USE `desafio_selecao`;

-- Copiando estrutura para tabela desafio_selecao.respostas_diagnostico
CREATE TABLE IF NOT EXISTS `respostas_diagnostico` (
  `id_resposta` int(11) N

Como se pode notar no print acima, é necessário arrumar o encoding do dump de sql. É o que fazemos abaixo

In [11]:
with open(dump_path, 'rb') as f:
    bin_sql_instructions = f.read()

In [12]:
sql_instructions = bin_sql_instructions.decode('utf-8')

#### Limpeza das instruções SQL

Nas células abaixo realizamos a limpeza da instrução SQL para que ela se adeque ao idioma implementado no SQLite.

In [13]:
sql_instructions = sql_instructions.replace('`', "'")

In [14]:
def remove_comments(sql_instructions):
    
    text_comment = ("COMMENT '")
    
    while True:
        
        ini = sql_instructions.find(text_comment)
        if ini < 0:
            break
        fim = sql_instructions[ini+len(text_comment):].find("'")
        
        comentario = sql_instructions[ini:ini+fim+len(text_comment)+1]
        
        sql_instructions = sql_instructions.replace(comentario, '')
    
    return sql_instructions

In [15]:
sql_instructions = remove_comments(sql_instructions)

In [16]:
def remove_collate(sql_instructions):
    
    collate_cmd = 'COLLATE utf8_unicode_ci'
    
    while True:
        found = sql_instructions.find(collate_cmd)
        if found<0:
            break
        sql_instructions = sql_instructions.replace(collate_cmd, '')
        
    return sql_instructions

In [17]:
sql_instructions = remove_collate(sql_instructions)

In [18]:
sql_instructions =sql_instructions.replace('ENGINE=InnoDB', '')

### Identificando comandos Create table e Insert values

Nas células abaixo, parseamos o dump SQL limpo para isolar as instruções Create Table e Insert values.

Em seguida, executamos essas instruções, criando efetivamente o banco de dados e o populando com dados.

In [19]:
create_table_start = sql_instructions.find('CREATE TABLE')
fim_create_table = 'DEFAULT CHARSET=utf8'
create_table_command = sql_instructions[create_table_start:sql_instructions.find(fim_create_table)]


Como se pode ver, a instrução create table está de acordo com o dicionario enviado.

Notamos também que os dados possuem chave única composta pelas variáveis id resposta e ano do diagnóstico.

In [20]:
print(create_table_command)

CREATE TABLE IF NOT EXISTS 'respostas_diagnostico' (
  'id_resposta' int(11) NOT NULL ,
  'ano_diagnostico' int(11) NOT NULL ,
  'data_submissao' datetime DEFAULT NULL ,
  'orgao' text  DEFAULT NULL ,
  'tipo_orgao' text  DEFAULT NULL ,
  'qtd_equipe' decimal(10,0) DEFAULT NULL ,
  'utiliza_metodologia' int(1) DEFAULT NULL ,
  'desktop_proprio' text  DEFAULT NULL ,
  'desktop_locado' text  DEFAULT NULL ,
  'desktop_proprio_antigo' text  DEFAULT NULL ,
  PRIMARY KEY ('id_resposta','ano_diagnostico')
)  


In [21]:
dicionario

,nome do campo,tipo,aceita nulos?,descrição
0,id_resposta,INT(11),NÃO,Chave única para cada resposta do formulário
1,ano_diagnostico,INT(11),NÃO,Ano de realização da pesquisa
2,data_submissao,DATETIME,SIM,Data de envio da resposta
3,orgao,TEXT,SIM,Selecione o Órgão onde trabalha:
4,tipo_orgao,TEXT,SIM,Classificação quanto ao tipo de órgão (Pode se...
5,qtd_equipe,"DECIMAL(10,0)",SIM,Quantas pessoas trabalham de forma dedicada à ...
6,utiliza_metodologia,INT(1),SIM,É utilizada alguma metodologia de gerenciament...
7,desktop_proprio,TEXT,SIM,Preencha os ítens abaixo de acordo com os quan...
8,desktop_locado,TEXT,SIM,Preencha os ítens abaixo de acordo com os quan...
9,desktop_proprio_antigo,TEXT,SIM,Do total de equipamentos PRÓPRIOS informados n...


Agora vamos criar o banco de dados e executar os comandos.

In [22]:
con = con = sqlite3.connect(db_path)

In [23]:
cur = con.cursor()

In [24]:
cur.execute(create_table_command)

In [25]:
con.commit()

In [26]:
insert_command_ini = sql_instructions.find('INSERT INTO')

insert_command_end = sql_instructions.find("*!40000 ALTER TABLE 'respostas_diagnostico' ENABLE KEYS */;\r\n\r\n/*!40101 SET SQL_MODE=IFNULL(@OLD_SQL_MODE, '')")

insert_command = sql_instructions[insert_command_ini:insert_command_end]

In [27]:
cur.execute(insert_command)

In [28]:
con.commit()

#### Checagem

Por fim, fazemos uma query select para verificar se os dados foram carregados com sucesso

In [29]:
cur.execute('''
            SELECT 
            name
        FROM 
            sqlite_schema
        WHERE 
            type ='table' AND 
            name NOT LIKE 'sqlite_%';''')

In [30]:
cur.fetchall()

[('respostas_diagnostico',)]

In [31]:
pd.read_sql('SELECT * FROM respostas_diagnostico', con)

,id_resposta,ano_diagnostico,data_submissao,orgao,tipo_orgao,qtd_equipe,utiliza_metodologia,desktop_proprio,desktop_locado,desktop_proprio_antigo
0,5,2017,2017-04-20 10:39:54,SUBPR,Subprefeitura,3.0,0.0,152,0,107
1,8,2019,2019-04-12 10:55:24,SMS,Secretaria,58.0,0.0,12500,0,6927.0000000000
2,9,2019,2019-04-05 15:57:10,SUBSA,Subprefeitura,3.0,NaN,36,90,0.0000000000
3,10,2017,2017-05-17 09:13:17,IPREM,Administração Indireta,6.0,0.0,200,0,0
4,10,2019,2019-03-25 14:40:06,SUBST,Subprefeitura,3.0,NaN,150,0,150.0000000000
...,...,...,...,...,...,...,...,...,...,...
302,396,2017,None,None,None,NaN,NaN,None,None,None
303,397,2017,None,None,None,NaN,NaN,None,None,None
304,398,2017,None,A3,None,40.0,0.0,None,None,None
305,399,2017,None,A3,None,NaN,NaN,,,


In [32]:
con.close()